In [2]:
# install and load packages
import pandas as pd
import mechanize
from http.cookiejar import CookieJar
from bs4 import BeautifulSoup as soup
import html2text
import os
import glob

In [5]:
# Load xlsx dataset
Fixed = pd.ExcelFile("..//Datasets/NEW/TailNumber_RW_FW_030921.xlsx", engine='openpyxl')

In [6]:
# excel content
Fixed.sheet_names

['FW', 'RW']

In [7]:
# Extract "Data" sheet
FW = Fixed.parse(0)

# Extract tail numbers
tails = list(FW.Tail_Num_FW.drop_duplicates().dropna().str.replace(' ', '', regex=True))
len(tails)

333

In [13]:
tails[1]

'N149CF'

In [ ]:
# login to the website
# Browser
br = mechanize.Browser()

# Cookie Jar
cj = CookieJar()
br.set_cookiejar(cj)

# Browser options
br.set_handle_equiv(True)
br.set_handle_gzip(True)
br.set_handle_redirect(True)
br.set_handle_referer(True)
br.set_handle_robots(False)
br.set_handle_refresh(mechanize._http.HTTPRefreshProcessor(), max_time=1)

br.addheaders = [('User-agent', 'Chrome')]

# The site we will navigate into, handling it's session
br.open('https://flightaware.com/account/session')

# View available forms
for f in br.forms():
    print (f)

# Select the second (index one) form (the first form is a search query box)
br.select_form(nr=7)

# User credentials
br.form["flightaware_username"] = 'kismailo'
br.form['flightaware_password'] = 'gwynbleidd'

# Login
br.submit()

In [176]:
tail_number = []
date = []
aircraft = []
origin = []
origin_location = []
destination = []
destination_location = []
departure = []
arrival = []
duration = []

for t in tails:
    url = 'https://flightaware.com/live/flight/' + t + '/history/320'
    page_html = br.open(url).read()
    page_soup = soup(page_html, 'html.parser')
    table = page_soup.findAll("tr", {"class":["smallActiverow1 rowClickTarget", "smallActiverow2 rowClickTarget"]})

    for i in range(0,len(table)):
        tail_number += [t]
        date += table[i].find_all("td")[0].a
        aircraft += [table[i].find_all("td")[1].text]
        origin += [table[i].find_all("td")[2].text]
        if len(table[i].find_all("td")[2].text) != 0:
            origin_location += [table[i].find_all("td")[2].span['title']]
        else:
            origin_location += ["Unknown"]
        destination += [table[i].find_all("td")[3].text]
        if len(table[i].find_all("td")[3].text) != 0:
            destination_location += [table[i].find_all("td")[3].span['title']]
        else:
            destination_location += ["Unknown"]
        departure += [table[i].find_all("td")[4].text]
        arrival += [table[i].find_all("td")[5].text]
        duration += [table[i].find_all("td")[6].text]  
            
flight_data = pd.DataFrame(list(zip(tail_number, date, aircraft, origin, origin_location, destination, destination_location, departure, arrival, duration)), 
                    columns=['tail_number','date', 'aircraft', 'origin', 'origin_location', 'destination', 'destination_location', 'departure', 'arrival', 'duration'])

In [178]:
# merge names, and export data
flight_data.to_csv('..//Datasets/FW_flights.csv', index=False)

In [170]:
Tail_number = []
Summary = []
Owner = []
Airworthiness_Class = []
Serial_Number = []
Engine = []
Weight = []
Speed = []
Mode_S_Code = []
Status = []
Certificate_Issue_Date = []
Airworthiness_Date = []
Last_Action_Date = []
Expiration = []
Registry_Source = []

for t in tails:
    url_2 = 'https://flightaware.com/resources/registration/' + t
    page_html_2 = br.open(url_2).read()
    page_soup_2 = soup(page_html_2, 'html.parser')
    table_2 = page_soup_2.findAll("div", {"class":"medium-3 columns"})
    
    if len(table_2) != 0: 
        Tail_number += [t]
        Summary += [table_2[0].text.replace('\n', '').replace('\t', '')]
        Owner += [table_2[1].text.replace('\n', '').replace('\t', '')]
        Airworthiness_Class += [table_2[2].text]
        Serial_Number += [table_2[3].text]
        Engine += [table_2[4].text.replace(' ', '')]
        Weight += [table_2[5].text]
        Speed += [table_2[6].text]
        Mode_S_Code += [table_2[7].text]
        Status += [table_2[8].text]
        Certificate_Issue_Date += [table_2[9].text]
        Airworthiness_Date += [table_2[10].text]
        Last_Action_Date += [table_2[11].text]
        Expiration += [table_2[12].text]
        Registry_Source += [table_2[13].text] 
    else:
        Tail_number += [t]
        Summary += [None]
        Owner += [None]
        Airworthiness_Class += [None]
        Serial_Number += [None]
        Engine += [None]
        Weight += [None]
        Speed += [None]
        Mode_S_Code += [None]
        Status += [None]
        Certificate_Issue_Date += [None]
        Airworthiness_Date += [None]
        Last_Action_Date += [None]
        Expiration += [None]
        Registry_Source += [None]

In [174]:
services_data = pd.DataFrame(list(zip(Tail_number,Summary,Owner,Airworthiness_Class,Serial_Number,Engine,Weight,Speed,Mode_S_Code,Status,
                      Certificate_Issue_Date,Airworthiness_Date,Last_Action_Date,Expiration,Registry_Source)),
            columns=['Tail_number','Summary','Owner','Airworthiness_Class','Serial_Number','Engine','Weight', 
                     'Speed','Mode_S_Code','Status','Certificate_Issue_Date','Airworthiness_Date', 'Last_Action_Date',
                    'Expiration','Registry_Source'])

In [175]:
services_data.to_csv('..//Datasets/FW_services_info.csv', index=False)

In [296]:
# # tail number web scraping
# def tail_numbers(tail):
#     tail_number = []
#     date = []
#     aircraft = []
#     origin = []
#     origin_location = []
#     destination = []
#     destination_location = []
#     departure = []
#     arrival = []
#     duration = []

#     for t in tail:
#         url = 'https://flightaware.com/live/flight/' + t + '/history/320'
#         page_html = br.open(url).read()
#         page_soup = soup(page_html, 'html.parser')
#         table = page_soup.findAll("tr", {"class":["smallActiverow1 rowClickTarget", "smallActiverow2 rowClickTarget"]})

#         for i in range(0,len(table)):
#             tail_number += [t]
#             date += table[i].find_all("td")[0].a
#             aircraft += [table[i].find_all("td")[1].text]
#             origin += [table[i].find_all("td")[2].text]
#             origin_location += [table[i].find_all("td")[2].span['title']]
#             destination += [table[3].find_all("td")[3].text]
#             destination_location += [table[i].find_all("td")[3].span['title']]
#             departure += [table[i].find_all("td")[4].text]
#             arrival += [table[i].find_all("td")[5].text]
#             duration += [table[i].find_all("td")[6].text]  
            
#             date += table[i].find_all("td")[0].a
#             aircraft += table[i].find_all("td")[1]
#             origin += table[i].find_all("td")[2].span
#             if len(table[i].find_all("td")[3]) != 0:
#                 destination += table[i].find_all("td")[3].span
#             if len(table[i].find_all("td")[3]) == 0:
#                 destination += ["Unknown"]
#             if len(table[i].find_all("td")[4]) == 1:
#                 departure += [table[i].find_all("td")[4].em]
#             if len(table[i].find_all("td")[4]) == 2:
#                 departure += [table[i].find_all("td")[4]]
#             if table[i].find_all("td")[5].em != 'NoneType':
#                 arrival += [table[i].find_all("td")[5]]
#             if table[i].find_all("td")[5].em == 'NoneType':
#                 arrival += table[i].find_all("td")[5].em
#             duration += table[i].find_all("td")[6]

#     data = pd.DataFrame(list(zip(tail_number, dates, aircraft, origin, origin_location, destination, destination_location, departure, arrival, duration)), 
#                     columns=['tailnumber','dates', 'aircraft', 'origin', 'origin_location', 'destination', 'destination_location', 'departure', 'arrival', 'duration'])
#     return(data)

In [ ]:
# # apply function
# data = tail_numbers(tails)